In [1]:
from lotc import FileBinaryClassifierTrainer, LongTextEncoder

long_text_encoder = LongTextEncoder(
    max_length=2048,
    chunk_size=512,
    overlapping=128
)

trainer = FileBinaryClassifierTrainer(
    long_text_encoder=long_text_encoder,
    batch_size=4,
    train_ratio=0.9
)

In [2]:
from lotc.util import get_files, read_file

pos_data_path = './data/pos'
neg_data_path = './data/neg'
pos_data = [read_file(x) for x in get_files(pos_data_path)]
neg_data = [read_file(x) for x in get_files(neg_data_path)]
model = trainer.train(pos_data[:2], neg_data[:2], num_epochs=20, learning_rate=2e-5)

[DEBUG] 2025-04-29 02:15:31,787 lotc.embedding : Embedding "#章节 01某天喝多，我把不苟言笑，禁欲系的老板当成了白月光。一觉醒来，我紧张问：「昨天……我们……」他答：「怎么，江秘书不打算负责？」我觉得我要死了，肯定要被炒鱿鱼了，可老板却说放我一周的假让我好好休息休息。一周后他把我堵墙角，问道：「给了你一周时间，想...".
Token indices sequence length is longer than the specified maximum sequence length for this model (614 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (743 > 512). Running this sequence through the model will result in indexing errors
[DEBUG] 2025-04-29 02:15:33,409 lotc.embedding : Embedding "#章节 01白月光一回来，陆景盛立马跟我提了离婚，我把孕检单塞进包包，强忍着泪意签下离婚协议：「既然这样，那我们好聚好散。」房子，车子，钱，资源……他都可以给我，唯独名分和爱，他永远只能给她。1、结婚三周年纪念日，当我满心欢喜想把怀孕的消息告诉他时，他先递...".
[DEBUG] 2025-04-29 02:15:34,978 lotc.embedding : Embedding "《一介替身》人人嫉妒我只是一介替身，却能稳坐探花郎夫人之位。闻言我内心苦涩，表面仍旧操持着偌大的沈府。我意外失去孩子的那日，沈尧瑾还在花楼之上搂着美娇娘风花雪月。我主动送去了那纸和离书。「你不过是月娘的替身罢了，若不是你有几分像她，我怎么会娶你？」「替身就...".
[DEBUG] 2025-04-29 02:15:36,480 lotc.embedding : Embed

In [3]:
model.save()

LogisticRegression(
  (fc1): Linear(in_features=3072, out_features=1024, bias=True)
  (fc1_to_fc4_res): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=1024, out_features=768, bias=True)
  (fc3): Linear(in_features=768, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=1, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.05)
)

In [4]:
# model = model.load()
print(long_text_encoder.encode('这是一个测试文本.'))
model.predict(long_text_encoder.encode('这是一个测试文本.').squeeze())

[DEBUG] 2025-04-29 02:15:38,712 lotc.embedding : Embedding "这是一个测试文本.".
[DEBUG] 2025-04-29 02:15:40,287 lotc.embedding : Embedding "这是一个测试文本.".


tensor([-0.0751,  0.4202,  0.5091,  ..., -0.2156,  0.4021, -0.1609])


tensor([0.5003])